1. python3 -m venv prototype-virtual-env 으로 가상환경 생성   
2. source prototype-virtual-env/bin/activate
3. pip install --upgrade pip + pip install ipykernel jupyter
4. 동일경로에 jupyter notebook 입력후 local vscode or cursor 에서 작업시작!  


In [47]:
!brew install cmake ffmpeg

==> Downloading https://formulae.brew.sh/api/formula.jws.json
==> Downloading https://formulae.brew.sh/api/cask.jws.json
To reinstall 4.0.1, run:
  brew reinstall cmake
To reinstall 7.1.1_2, run:
  brew reinstall ffmpeg


In [48]:
# 📥 whisper.cpp 클론
!git clone https://github.com/ggerganov/whisper.cpp.git

# 📁 디렉토리 이동
%cd whisper.cpp

# 🔨 CMake로 빌드 M4 맥북에서 CPU로 돌려볼려면 이거 써주세용
!cmake -B build
!cmake --build build --config Release

# 다시 디렉토리 밖으로 이동
%cd ..

fatal: destination path 'whisper.cpp' already exists and is not an empty directory.
/Users/aza1200/Desktop/RAG/whisper.cpp
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: arm64
-- Including CPU backend
-- Accelerate framework found
-- Could NOT find OpenMP_C (missing: OpenMP_C_FLAGS OpenMP_C_LIB_NAMES) 
-- Could NOT find OpenMP_CXX (missing: OpenMP_CXX_FLAGS OpenMP_CXX_LIB_NAMES) 
-- Could NOT find OpenMP (missing: OpenMP_C_FOUND OpenMP_CXX_FOUND) 
CMake Warning at ggml/src/ggml-cpu/CMakeLists.txt:63 (message):
  OpenMP not

In [49]:
!pip install huggingface_hub
!pip install -U langchain
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install -U faiss-cpu
!pip install -U gdown

In [ ]:
import os
import json
import gdown
import shutil
import subprocess
from getpass import getpass
from langchain.chains import RetrievalQA
from huggingface_hub import hf_hub_download
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [51]:
# 파일 이름 설정
LECTURE_FILE_NAME = "01"
MODEL_NAME = "ggml-small.bin"

os.makedirs("./AX_Data", exist_ok=True)
os.makedirs(f"./Sound/{LECTURE_FILE_NAME}", exist_ok=True)
os.makedirs(f"./Text/{LECTURE_FILE_NAME}", exist_ok=True)

# 🔽 모델 다운로드
MODEL_PATH = hf_hub_download(
    repo_id="ggerganov/whisper.cpp",
    filename=MODEL_NAME
)
print(f"모델 다운로드 완료 : {MODEL_PATH}")

모델 다운로드 완료 : /Users/aza1200/.cache/huggingface/hub/models--ggerganov--whisper.cpp/snapshots/5359861c739e955e79d9a303bcbc70fb988958b1/ggml-small.bin


In [52]:
video_path = f"./AX_Data/{LECTURE_FILE_NAME}.mp4"
# Google Drive에서 영상 다운로드
file_id = "1h9lxMPkahHB-dn7p4QBXkMwOZpgVaoSB"  # Google Drive 공유 링크에서 file_id 추출
url = f"https://drive.google.com/uc?id={file_id}"

# 다운로드
gdown.download(url, video_path, quiet=False)

print(f"✅ 영상 다운로드 완료: {video_path}")

Downloading...
From (original): https://drive.google.com/uc?id=1h9lxMPkahHB-dn7p4QBXkMwOZpgVaoSB
From (redirected): https://drive.google.com/uc?id=1h9lxMPkahHB-dn7p4QBXkMwOZpgVaoSB&confirm=t&uuid=c14dc6c5-2e22-458f-802d-e3587f360faf
To: /Users/aza1200/Desktop/RAG/AX_Data/01.mp4
100%|████████████████████████████████████████| 495M/495M [00:06<00:00, 72.0MB/s]

✅ 영상 다운로드 완료: ./AX_Data/01.mp4


In [53]:
chunk_output_dir = f"./Sound/{LECTURE_FILE_NAME}"

os.system(
    f"ffmpeg -y -i {video_path} "
    f"-vn -ar 16000 -ac 1 "
    f"-af silenceremove=start_periods=1:start_duration=1:start_threshold=-45dB:"
    f"stop_periods=-1:stop_duration=1:stop_threshold=-45dB,"
    f"aresample=async=1 "
    f"-f segment -segment_time 120 -c:a pcm_s16le "
    f"{chunk_output_dir}/chunk_%03d.wav"
)

print("✅ 무음 제거 + 2분 자르기 한 번에 완료")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

✅ 무음 제거 + 2분 자르기 한 번에 완료


[segment @ 0x127e0ea50] Opening './Sound/01/chunk_040.wav' for writing
[segment @ 0x127e0ea50] Opening './Sound/01/chunk_041.wav' for writing
[out#0/segment @ 0x600000280180] video:0KiB audio:155851KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: unknown
size=N/A time=01:23:07.21 bitrate=N/A speed=2.09e+03x    


In [54]:
def whisper_cpp_transcribe(file_name, model_name):
    model_label = model_name.replace(".bin", "")
    audio_path = f"./Sound/{LECTURE_FILE_NAME}/{file_name}.wav"
    output_file_prefix = f"./Text/{LECTURE_FILE_NAME}/{file_name}_{model_label}"

    os.makedirs(f"./Text/{LECTURE_FILE_NAME}", exist_ok=True)

    cmd = [
        "./whisper.cpp/build/bin/whisper-cli",
        "-m", MODEL_PATH,
        "-f", audio_path,
        "-l", "ko",
        "-oj",
        "-of", output_file_prefix,
    ]

    subprocess.run(cmd)
    print(f"✅ {file_name} 처리 완료 → {output_file_prefix}.json")

In [55]:
# ✅ .wav 파일만 필터링
sound_dir = f"./Sound/{LECTURE_FILE_NAME}"
chunk_files = sorted([
    f for f in os.listdir(sound_dir)
    if f.endswith(".wav")
])

chunk_count = len(chunk_files)
print("📦 chunk count:", chunk_count)
print("🎵 chunk files:", chunk_files[:5], "...")

# ✅ 병합용 경로 설정
model_label = MODEL_NAME.replace(".bin", "")
output_merged_path = f"./Text/{LECTURE_FILE_NAME}/{LECTURE_FILE_NAME}_merged_{model_label}.json"
merged_segments = []

# ✅ 42개의 chunk 처리
for i in range(chunk_count):
    chunk_id = f"chunk_{i:03d}"

    # whisper 처리 (여기 함수는 기존 그대로 사용)
    whisper_cpp_transcribe(chunk_id, MODEL_NAME)

    # 결과 JSON 읽기
    json_path = f"./Text/{LECTURE_FILE_NAME}/{chunk_id}_{model_label}.json"
    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_segments.extend(data.get("transcription", []))

# ✅ 최종 병합된 JSON 저장
with open(output_merged_path, "w", encoding="utf-8") as f:
    json.dump({"transcription": merged_segments}, f, indent=2, ensure_ascii=False)

print(f"✅ 전체 병합 완료 → {output_merged_path}")

whisper_init_from_file_with_params_no_state: loading model from '/Users/aza1200/.cache/huggingface/hub/models--ggerganov--whisper.cpp/snapshots/5359861c739e955e79d9a303bcbc70fb988958b1/ggml-small.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 0
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_init_with_params_no_state: devices    = 3
whisper_init_with_params_no_state: backends   = 3
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51865
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 768
whisper_model_load: n_audio_head  = 12
whisper_model_load: n_audio_layer = 12
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 768
whisper_model_load: n_text_head   = 12
whisper_model_load: n_text_layer  = 12
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr    

📦 chunk count: 42
🎵 chunk files: ['chunk_000.wav', 'chunk_001.wav', 'chunk_002.wav', 'chunk_003.wav', 'chunk_004.wav'] ...


whisper_model_load: model size    =  487.01 MB
whisper_backend_init_gpu: using Metal backend
ggml_metal_init: allocating
ggml_metal_init: found device: Apple M4 Pro
ggml_metal_init: picking default device: Apple M4 Pro
ggml_metal_load_library: using embedded metal library
ggml_metal_init: GPU name:   Apple M4 Pro
ggml_metal_init: GPU family: MTLGPUFamilyApple9  (1009)
ggml_metal_init: GPU family: MTLGPUFamilyCommon3 (3003)
ggml_metal_init: GPU family: MTLGPUFamilyMetal3  (5001)
ggml_metal_init: simdgroup reduction   = true
ggml_metal_init: simdgroup matrix mul. = true
ggml_metal_init: has residency sets    = true
ggml_metal_init: has bfloat            = true
ggml_metal_init: use bfloat            = false
ggml_metal_init: hasUnifiedMemory      = true
ggml_metal_init: recommendedMaxWorkingSetSize  = 38654.71 MB
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_in


[00:00:00.000 --> 00:00:20.980]   (인터뷰)
[00:00:20.980 --> 00:00:25.600]   혹시 그 여러분 젠킨스라는 투를 해본 적이 있다
[00:00:25.600 --> 00:00:27.440]   젠킨스 써본 적 있다
[00:00:27.440 --> 00:00:29.640]   (인터뷰)
[00:00:29.640 --> 00:00:32.480]   혹시 써보실 때 어느 정도 쓰시니 까지 써보셨어요
[00:00:32.480 --> 00:00:36.160]   그냥 사용자 입장에서 파이프라인이 짜는 정도?
[00:00:36.160 --> 00:00:39.580]   짜고 젠킨스로 그러면 뭘 하셨어요 혹시?
[00:00:39.580 --> 00:00:40.740]   (인터뷰)
[00:00:40.740 --> 00:00:44.720]   씨아이만 했어요 젠킨스로
[00:00:44.720 --> 00:00:47.160]   저희는 어제 컨테이너 빌드 를 했기 때문에
[00:00:47.160 --> 00:00:50.040]   오늘은 rgo cd로 별풀할 건데
[00:00:50.040 --> 00:00:52.920]   kt 안에서 이제 vmv4 대상으로는
[00:00:52.920 --> 00:00:55.440]   cd2로 젠킨스를 쓰고 있거든요
[00:00:55.440 --> 00:01:06.560]   그래서 그래서 이제 저희 개발맥을 써서
[00:01:06.560 --> 00:01:08.960]   간단하게 이제 토커업을 해서 이미지 가져와서 안에
[00:01:08.960 --> 00:01:12.520]   어떻게 부성되어 있고 어떻게 쓰는 지 정도만 확인을 하고
[00:01:12.520 --> 00:01:16.600]   그 다음에 저희가 교육용으로 구성해서 rgo cd로 넘어갈게요
[00:01:16.600 --> 00:01:22.240]   그러면 다들 토커업을 쓸 수 있는 개발맥을 가져와 보시고
[00:01:2

output_json: saving output to './Text/01/chunk_000_ggml-small.json'

whisper_print_timings:     load time =   178.23 ms
whisper_print_timings:     fallbacks =   0 p /   0 h
whisper_print_timings:      mel time =    28.44 ms
whisper_print_timings:   sample time =   307.59 ms /  1984 runs (     0.16 ms per run)
whisper_print_timings:   encode time =   614.41 ms /     5 runs (   122.88 ms per run)
whisper_print_timings:   decode time =    35.53 ms /     9 runs (     3.95 ms per run)
whisper_print_timings:   batchd time =  1803.40 ms /  1953 runs (     0.92 ms per run)
whisper_print_timings:   prompt time =    83.31 ms /   681 runs (     0.12 ms per run)
whisper_print_timings:    total time =  3136.04 ms
ggml_metal_free: deallocating
whisper_init_from_file_with_params_no_state: loading model from '/Users/aza1200/.cache/huggingface/hub/models--ggerganov--whisper.cpp/snapshots/5359861c739e955e79d9a303bcbc70fb988958b1/ggml-small.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_i

...

output_json: saving output to './Text/01/chunk_039_ggml-small.json'

whisper_print_timings:     load time =   183.90 ms
whisper_print_timings:     fallbacks =   1 p /   3 h
whisper_print_timings:      mel time =    28.46 ms
whisper_print_timings:   sample time =   162.87 ms /   736 runs (     0.22 ms per run)
whisper_print_timings:   encode time =   689.48 ms /     6 runs (   114.91 ms per run)
whisper_print_timings:   decode time =   793.40 ms /   208 runs (     3.81 ms per run)
whisper_print_timings:   batchd time =   567.96 ms /   512 runs (     1.11 ms per run)
whisper_print_timings:   prompt time =    39.27 ms /   206 runs (     0.19 ms per run)
whisper_print_timings:    total time =  2522.52 ms
ggml_metal_free: deallocating
whisper_init_from_file_with_params_no_state: loading model from '/Users/aza1200/.cache/huggingface/hub/models--ggerganov--whisper.cpp/snapshots/5359861c739e955e79d9a303bcbc70fb988958b1/ggml-small.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_i


[00:00:00.000 --> 00:00:03.720]   여기 이제 우리 오버니제이션메인으로 들어오셔가지고
[00:00:03.720 --> 00:00:07.000]   배포 스토리를 새로 하나 만들거에요. 근데
[00:00:07.000 --> 00:00:09.160]   어 이제
[00:00:09.160 --> 00:00:11.620]   저번에 보니까 이제 소스코도 안에
[00:00:11.620 --> 00:00:14.600]   포어네티스 형상들이 있는 경우가 있었어요.
[00:00:14.600 --> 00:00:18.900]   이분은 없으신?
[00:00:18.900 --> 00:00:23.500]   혹시 배포 뭘로 하셨어요? 그때? 각자 배포 해보셨을 때?
[00:00:23.500 --> 00:00:28.580]   포어네티스 배포 할 때? 헬렌 배포 이런거 할 때?
[00:00:28.580 --> 00:00:30.960]   아 그래요? 그럼 거기에 배포 형상들이 있어요.
[00:00:30.960 --> 00:00:34.900]   각자 하나씩? 그러면 그거 활용하시면 될 것 같고
[00:00:34.900 --> 00:00:39.300]   동일하게 해야되는데 우리는
[00:00:39.300 --> 00:00:44.120]   기돕스로, HTTPS로 배포를 해볼게요. 그러면
[00:00:44.120 --> 00:00:47.360]   지금 어제 이렇게 올려주신 랩코스토리는
[00:00:47.360 --> 00:00:52.260]   소스코드 랩코스토리라고 하는거에요. 그러니까 어플리케이션 개발을 위한 소스코드들만
[00:00:52.260 --> 00:00:53.880]   저장을 해둘거고
[00:00:53.880 --> 00:00:56.540]   우리는 이제 기돕스 랩포를 하나 만들어야 되죠. 여기
[00:00:56.540 --> 00:00:59.760]   ARGO CD 전용으로 기돕스 랩포를 하나 만들거에요. 그래서
[00:00:59.7

output_json: saving output to './Text/01/chunk_040_ggml-small.json'

whisper_print_timings:     load time =   192.72 ms
whisper_print_timings:     fallbacks =   0 p /   0 h
whisper_print_timings:      mel time =    28.80 ms
whisper_print_timings:   sample time =   401.99 ms /  2524 runs (     0.16 ms per run)
whisper_print_timings:   encode time =   591.95 ms /     5 runs (   118.39 ms per run)
whisper_print_timings:   decode time =    80.56 ms /    19 runs (     4.24 ms per run)
whisper_print_timings:   batchd time =  2327.57 ms /  2483 runs (     0.94 ms per run)
whisper_print_timings:   prompt time =    96.01 ms /   816 runs (     0.12 ms per run)
whisper_print_timings:    total time =  3793.90 ms
ggml_metal_free: deallocating
whisper_init_from_file_with_params_no_state: loading model from '/Users/aza1200/.cache/huggingface/hub/models--ggerganov--whisper.cpp/snapshots/5359861c739e955e79d9a303bcbc70fb988958b1/ggml-small.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_i


[00:00:00.000 --> 00:00:02.000]   전화해 주시는 거 같아요.
[00:00:02.000 --> 00:00:05.000]   (댓글 읽는 중)
[00:00:05.000 --> 00:00:07.000]   아 거기 있어요?
[00:00:07.000 --> 00:00:09.000]   (댓글 읽는 중)
[00:00:09.000 --> 00:00:11.000]   알겠습니다.
[00:00:11.000 --> 00:00:13.000]   (댓글 읽는 중)
[00:00:13.000 --> 00:00:17.000]   그래서 래퍼 하나씩 만들어 보네요.
[00:00:17.000 --> 00:00:20.000]   (댓글 읽는 중)
[00:00:20.000 --> 00:00:22.000]   (댓글 읽는 중)
[00:00:22.000 --> 00:00:25.000]   (댓글 읽는 중)
[00:00:25.000 --> 00:00:28.000]   (댓글 읽는 중)
[00:00:28.000 --> 00:00:30.000]   (댓글 읽는 중)
[00:00:30.000 --> 00:00:32.000]   (댓글 읽는 중)
[00:00:32.000 --> 00:00:34.000]   (댓글 읽는 중)
[00:00:34.000 --> 00:00:36.000]   (댓글 읽는 중)
[00:00:36.000 --> 00:00:38.000]   (댓글 읽는 중)
[00:00:38.000 --> 00:00:40.000]   (댓글 읽는 중)
[00:00:40.000 --> 00:00:42.000]   (댓글 읽는 중)
[00:00:42.000 --> 00:00:44.000]   (댓글 읽는 중)
[00:00:44.000 --> 00:00:46.000]   (댓글 읽는 중)
[00:00:46.000 --> 00:00:48.000]   (댓글 읽는 중)
[00:00:48.000 --> 00:00:50.000]   (댓글 읽는 중)
[00:00:50.000 --> 0

output_json: saving output to './Text/01/chunk_041_ggml-small.json'

whisper_print_timings:     load time =   199.82 ms
whisper_print_timings:     fallbacks =   1 p /   0 h
whisper_print_timings:      mel time =    16.60 ms
whisper_print_timings:   sample time =   262.86 ms /  1679 runs (     0.16 ms per run)
whisper_print_timings:   encode time =   355.43 ms /     3 runs (   118.48 ms per run)
whisper_print_timings:   decode time =    49.51 ms /    12 runs (     4.13 ms per run)
whisper_print_timings:   batchd time =  1785.70 ms /  1653 runs (     1.08 ms per run)
whisper_print_timings:   prompt time =    44.13 ms /   343 runs (     0.13 ms per run)
whisper_print_timings:    total time =  2781.29 ms
ggml_metal_free: deallocating


In [56]:
# STEP 1. Whisper JSON 불러오기
with open(f"./Text/{LECTURE_FILE_NAME}/{LECTURE_FILE_NAME}_merged_ggml-small.json", "r", encoding="utf-8") as f:
    data = json.load(f)

segments = data["transcription"]
segments[:100]

[{'timestamps': {'from': '00:00:00,000', 'to': '00:00:20,980'},
  'offsets': {'from': 0, 'to': 20980},
  'text': ' (인터뷰)'},
 {'timestamps': {'from': '00:00:20,980', 'to': '00:00:25,600'},
  'offsets': {'from': 20980, 'to': 25600},
  'text': ' 혹시 그 여러분 젠킨스라는 투를 해본 적이 있다'},
 {'timestamps': {'from': '00:00:25,600', 'to': '00:00:27,440'},
  'offsets': {'from': 25600, 'to': 27440},
  'text': ' 젠킨스 써본 적 있다'},
 {'timestamps': {'from': '00:00:27,440', 'to': '00:00:29,640'},
  'offsets': {'from': 27440, 'to': 29640},
  'text': ' (인터뷰)'},
 {'timestamps': {'from': '00:00:29,640', 'to': '00:00:32,480'},
  'offsets': {'from': 29640, 'to': 32480},
  'text': ' 혹시 써보실 때 어느 정도 쓰시니 까지 써보셨어요'},
 {'timestamps': {'from': '00:00:32,480', 'to': '00:00:36,160'},
  'offsets': {'from': 32480, 'to': 36160},
  'text': ' 그냥 사용자 입장에서 파이프라인이 짜는 정도?'},
 {'timestamps': {'from': '00:00:36,160', 'to': '00:00:39,580'},
  'offsets': {'from': 36160, 'to': 39580},
  'text': ' 짜고 젠킨스로 그러면 뭘 하셨어요 혹시?'},
 {'timestamps': {'from

In [57]:
# STEP 2. 전체 텍스트 이어붙이기
full_text = "\n".join([seg["text"] for seg in segments])
full_text[:100]

' (인터뷰)\n 혹시 그 여러분 젠킨스라는 투를 해본 적이 있다\n 젠킨스 써본 적 있다\n (인터뷰)\n 혹시 써보실 때 어느 정도 쓰시니 까지 써보셨어요\n 그냥 사용자 입장에서 파이프'

In [58]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

chunks = text_splitter.split_text(full_text)


In [63]:
os.environ["OPENAI_API_KEY"] = getpass("🔐 OpenAI API Key를 입력하세요: ")

In [64]:
# STEP 4. Document 리스트로 변환
docs = [Document(page_content=chunk, metadata={}) for chunk in chunks]

# STEP 5. Embedding + Vectorstore 생성
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)

In [76]:
# STEP 6. Retriever + QA Chain 구성
retriever = vectorstore.as_retriever(search_type="mmr")
retriever.search_kwargs.update({
    "k": 10,
    "fetch_k": 100,
    "maximal_marginal_relevance": True
})

# custom_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# 다음 문서를 참고하여 질문에 답하세요.

# 문서에 명확한 내용이 없으면 반드시 아래와 같이 대답하세요:
# '잘 모르겠습니다. 문서에 관련 정보가 없습니다.'

# 문서:
# {context}

# 질문:
# {question}
# """
# )

from langchain.callbacks import StdOutCallbackHandler


qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    callbacks=[StdOutCallbackHandler()]
    # chain_type_kwargs={"prompt": custom_prompt}
)

In [77]:
# STEP 7. 유저 질문 테스트
query = "젠킨스는 오픈소스 CI/CD 자동화 서버이다. 이거 맞아?"
result = qa(query)

print("💬 Question:", query)
print(result)
print("🧠 Answer:", result['result'])
print("📄 Source Metadata:", result['source_documents'])



> Entering new RetrievalQA chain...

> Finished chain.
💬 Question: 젠킨스는 오픈소스 CI/CD 자동화 서버이다. 이거 맞아?
{'query': '젠킨스는 오픈소스 CI/CD 자동화 서버이다. 이거 맞아?', 'result': '', 'source_documents': [Document(id='b7558aae-d869-48a8-92f6-ff99ea27d4d4', metadata={}, page_content='그런 경우에는 그냥 커버네티스를 쓴게 났습니다. 뭐 근데 이런 여러 가지 선택지가 장난점은 있지만 뭐가 좋고 나쁘다 라기보다는\n 본인의 서비스에 맞춰서 이제 사용하시면 될 것 같아요. 그리고 이제 저희가 기본적으로 지금까지 오늘 오전에 좀 vm 배포도구로 간단하게 젠킷을 썼죠.\n 젠킷으로 이제 대충 뭐가 교정되어 있는지까지 봤는데 이렇게 배포 자동화도구로 vm에다 배포하는 경우에 앤서블이라는 걸 많이 쓰고 있어요. 앤서블은\n 플레이북이라는 단위로 약간 이런거죠. 앤서블 하나\n 자다가 연동되어 있는 모든 VM에다가 한 번의 명령어를 때려서 안에 있는 OS 단 부터 해서\n 그 안에 있는 서버의 구성이라든가 그런 거를 한 번에 취할 수 있는 중앙화된 SSH 기반의 트리거든요.\n 이런 것도 있고, 오늘 다뤘던 진키스는 굉장히 유해가 잘 되어 있기 때문에\n 타입 프라인을 잘 짜면 앤선분처럼 잘 유용하게 쓸 수 있고요.\n 그 다음에 KT 앤섭프라이즈 환경에서는 저희가 기톱 액션으로 배포하지 않지만\n 그냥 이제 퍼블릿 환경에서는 기톱 액션으로도 배포할 수 있습니다. 이런 게 있고, 이해하시면 되고\n 컨테이너 같은 경우는 진키스로도 충분히 배폴할 수 있어요.\n 그래서 여러분들이 입사하시기 전에 제가 예전에 KT 내부의 개발자 플랫폼이라고 해서\n 데브스페이스라는 거 운영을 했는데 그때는 이제 여러분들의 애저에 배포하시겠지만\n KT 클라우드라는 그룹사가 있죠. KT 클라우드는 데이터센터로 운영하